---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [151]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [180]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    # Bear in mind that the data cleaning might result in trailing white space, if 2 is not followed correctly.

    univtowns = pd.read_csv('university_towns.txt', sep='\n', header = None)
    univtowns['RegionName'] = univtowns[0]
    univtowns['IsState'] = univtowns['RegionName'].str.contains('\[edit\]').astype(int)
    univtowns['RawState'] = univtowns['IsState']*univtowns['RegionName'].str.replace(r"\[.*\]",'')
    univtowns['RegionName'] = univtowns['RegionName'].str.replace(r"\(.*","").str.replace(r"\[.*\]",'')
    univtowns['RegionName'] = univtowns['RegionName'].map(lambda x: x.rstrip().lstrip())
    previousState=univtowns.iloc[0]['RawState']
    for index,row  in univtowns.iterrows():
        if (row['IsState']>0):
            previousState=univtowns.iloc[index]['RawState']
            univtowns.set_value(index, 'State', previousState)
        else:
            univtowns.set_value(index, 'State', previousState)
    univtowns.drop('RawState', axis=1, inplace=True)    
    univtowns.drop(univtowns.columns[0],axis=1,inplace=True)
    univtowns = univtowns[univtowns['IsState']==0]

    nd = univtowns['State'].to_frame()
    nd['RegionName'] = univtowns['RegionName']
    
    return nd

df = get_list_of_university_towns()
#print (df[df['State'].str.contains('\n') | df['RegionName'].str.contains('\n')])
#print(df[df['State'].str.contains('\s$',regex=True) | df['RegionName'].str.contains('\s$',regex=True)])
#output should be in both cases: #Empty DataFrame, #Columns: [State, RegionName] #Index: []

In [152]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    '''From Bureau of Economic Analysis, US Department of Commerce, 
    the GDP over time of the United States in current dollars 
    use the chained value in 2009 dollars), in quarterly intervals, in the file gdplev.xls. 
    For this assignment, only look at GDP data from the first quarter of 2000 onward.'''
    dfgdp = pd.read_excel('gdplev.xls',header=3, skiprows=2, skip_footer=0)
    dfgdp.drop(dfgdp.columns[[0,1,2,3,7]],axis=1,inplace=True)
    dfgdp.columns = ['YearQuarter', 'GDPcurrent', 'GDPchained']
    dfgdp['Year'] = (dfgdp['YearQuarter'].str.replace(r"q.*",'')) 
    #pd.to_datetime(dfgdp['YearQuarter'].str.replace(r"q.*",''))
    dfgdp = dfgdp[dfgdp['Year']> '1999'] #'1999-12-31']
    dfgdp['Quarter'] = (dfgdp['YearQuarter'].str.replace(r".*q",''))
    dfgdp = dfgdp.reset_index()
    #dfgdp = dfgdp.set_index(['Year','Quarter'])
    dfgdp.drop('index', axis=1, inplace=True)
    dfgdp.drop('YearQuarter', axis=1, inplace=True)
    
    '''A recession is defined as starting with 
    two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.'''
    recesstartYear=0
    recesstartQ=0
    previousGdp=0
    SearchStartRecession=True
    IsPreviousDecline=False
    for index,row  in dfgdp.iterrows():
        if (SearchStartRecession):
            if (row['GDPchained']<previousGdp):                
                if (IsPreviousDecline):
                    SearchStartRecession=False                    
                IsPreviousDecline=True
                if (SearchStartRecession):
                    recesstartYear=row['Year']
                    recesstartQ=row['Quarter']
            else:
                IsPreviousDecline=False
            previousGdp=dfgdp.iloc[index]['GDPchained']

    recesstart=recesstartYear+'q'+recesstartQ
    #print(recesstart)
    #print(dfgdp.tail(35).head(6))

    return recesstart
get_recession_start()

'2008q3'

In [156]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import re

    recesstartYearQ=get_recession_start()   
    startYear = re.sub(r"q.*",'', recesstartYearQ)
    startQ = re.sub(r".*q",'', recesstartYearQ)
    #--------------------------------------
    dfgdp = pd.read_excel('gdplev.xls',header=3, skiprows=2, skip_footer=0)
    dfgdp.drop(dfgdp.columns[[0,1,2,3,7]],axis=1,inplace=True)
    dfgdp.columns = ['YearQuarter', 'GDPcurrent', 'GDPchained']
    dfgdp['Year'] = (dfgdp['YearQuarter'].str.replace(r"q.*",'')) 
    #pd.to_datetime(dfgdp['YearQuarter'].str.replace(r"q.*",''))
    dfgdp = dfgdp[dfgdp['Year']>= startYear] #'1999-12-31'] #Include quarter!
    dfgdp['Quarter'] = (dfgdp['YearQuarter'].str.replace(r".*q",''))
    dfgdp = dfgdp.reset_index()
    dfgdp.drop('index', axis=1, inplace=True)
    dfgdp.drop('YearQuarter', axis=1, inplace=True)
    dfgdp.drop('GDPcurrent', axis=1, inplace=True)

    #print(dfgdp)
    '''A recession is defined as ending with two consecutive quarters of GDP growth.'''
    recessendYear=10000
    recessendQ=0
    previousGdp=dfgdp.iloc[0]['GDPchained']
    SearchEndRecession=True
    IsPreviousRise=False
    
    for index,row  in dfgdp.iterrows():
        if (SearchEndRecession):
            
            if (row['GDPchained']>previousGdp):
                recessendYear=row['Year']
                recessendQ=row['Quarter']
                if (IsPreviousRise):
                    SearchEndRecession=False                    
                IsPreviousRise=True
            else:
                IsPreviousRise=False
            previousGdp=dfgdp.iloc[index]['GDPchained']

    recessend=recessendYear+'q'+recessendQ
    return recessend
       
get_recession_end()

'2009q4'

In [157]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import re

    recesstartYearQ=get_recession_start()   
    startYear = re.sub(r"q.*",'', recesstartYearQ)
    startQ = re.sub(r".*q",'', recesstartYearQ)
    recessendYearQ=get_recession_end()   
    endYear = re.sub(r"q.*",'', recessendYearQ)
    endQ = re.sub(r".*q",'', recessendYearQ)
    #print(startYear)
    #print(endYear)
    #--------------------------------------
    dfgdp = pd.read_excel('gdplev.xls',header=3, skiprows=2, skip_footer=0)
    dfgdp.drop(dfgdp.columns[[0,1,2,3,7]],axis=1,inplace=True)
    dfgdp.columns = ['YearQuarter', 'GDPcurrent', 'GDPchained']
    dfgdp['Year'] = (dfgdp['YearQuarter'].str.replace(r"q.*",'')) 
    dfgdp = dfgdp[dfgdp['Year']>= startYear] #'1999-12-31'] #Include quarter!
    dfgdp = dfgdp[dfgdp['Year']<= endYear] 
    dfgdp['Quarter'] = (dfgdp['YearQuarter'].str.replace(r".*q",''))
    dfgdp = dfgdp.reset_index()
    dfgdp.drop('index', axis=1, inplace=True)
    dfgdp.drop('YearQuarter', axis=1, inplace=True)
    dfgdp.drop('GDPcurrent', axis=1, inplace=True)
    #print(dfgdp)
    
    #ndf= dfgdp['GDPchained']
    #print(dfgdp.info())
    iGDPmin = dfgdp['GDPchained'].idxmin()
    recessbottom=(dfgdp.iloc[iGDPmin]['Year'] + 'q' + dfgdp.iloc[iGDPmin]['Quarter'])
    #--------------------------------------
    return recessbottom

get_recession_bottom()

'2009q2'

In [148]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.    
    The resulting dataframe should have 67 columns, and 10,730 rows.  
    '''
    housingdf = pd.read_csv('City_Zhvi_AllHomes.csv', header = 0)
    #print(housingdf.iloc[0:5,123:129])
    
    startcol=(housingdf.columns.get_loc('2000-01'))
    endcol=(housingdf.columns.get_loc('2016-08'))+1
    ndf = housingdf.iloc[:, startcol:endcol]
    
    ndf = ndf.T
    ndf.index = pd.to_datetime(ndf.index) 
    #ndf = ndf.resample('90D').mean()
    #print(ndf.iloc[71:77,0:5]) #print(ndf.iloc[51:57,0:5])
    ndf = ndf.resample('Q', convention='end').apply(np.mean)
    #print(ndf.iloc[18:28,0:4])
    
    Qhousingdf=ndf.T.copy()
    #-------------------------------
    cols = []
    for year in range(2000, 2017):
        for quarter in range(1, 5):
            nname = (str(year) + 'q' + str(quarter))
            cols.append(nname)
    cols=cols[0:67]
               
    Qhousingdf.columns = cols
    
    
    colskeep=['State','RegionName'] #,'RegionID']
    infodf = housingdf[colskeep]
    Qhousingdf = pd.concat([Qhousingdf, infodf], axis=1)
    
    colsorder = colskeep + cols
    Qhousingdf = Qhousingdf[colsorder]
    Qhousingdf['State'].replace(states, inplace=True)
    
    Qhousingdf = Qhousingdf.set_index(['State','RegionName'])
    
    #Qhousingdf.drop('2016q4', axis=1, inplace=True)    
    Qhousingdf = Qhousingdf.sort_index()
    #Qhousingdf.to_csv('aQhousingdf.csv')
    #print(Qhousingdf.head())
    ''' xx '''
    #-------------------------------
    return Qhousingdf

df=convert_housing_data_to_quarters()
#print(df)
#print(df.iloc[6238:6240, 24:28])#print(df[df['State'].str.contains('Alas')].head())

In [337]:
def run_ttest():
    '''1 Merged university towns with all towns data to get a set that's common to both, i.e., university towns
    that exist in all towns.
    2 Subtracted this from the all towns data to get towns with no universities.
    3 Computed price ratio: quarter BEFORE recession start / recession bottom quarter.
    4 Computed the tuple requested. '''
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence.  '''
    
    
    recStart=get_recession_start()
    recBottom=get_recession_bottom()
    cols=[ recStart, recBottom]
    #print('Compare:' + recStart + ' vs ' + recBottom)
    
    hdf = convert_housing_data_to_quarters()
    iCStart=hdf.columns.get_loc(recStart)
    iCEnd=hdf.columns.get_loc(recBottom) +1
    
    df = hdf[cols]
    df.dropna(axis=0, how='all',inplace='True') #Drop!!!  #  print(len(df)) 
    
    ''' Hypothesis: University towns have their mean housing prices less effected by recessions. 
    Run a t-test to compare the ratio of the mean price of houses in university towns the quarter 
    before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)'''
    df['price_ratio']=df[recStart]/df[recBottom]
    df.drop(df.columns[[0,1]],axis=1,inplace=True)
    #print((df.info()))     
    #-----------------------------------------------
    ul = get_list_of_university_towns()
    ul.set_index(['State','RegionName'],inplace=True)
    
    dfUniv =pd.merge(ul, df, how = 'left', left_index='True', right_index='True')    
    
    dfNUniv = hdf.index.difference(dfUniv.index)#
    dfNUniv = pd.DataFrame(index=dfNUniv)
    dfNUniv = pd.merge(dfNUniv, df, how = 'left', left_index=True, right_index=True)
    
    dfUniv.dropna(axis=0,inplace='True')
    dfNUniv.dropna(axis=0,inplace='True')

    #print(dfUniv.info()) #.head())
    #print()
    #print(dfNUniv.info()) #head())
    #print( str(len(dfUniv)) + '-' +str(len(dfNUniv)) )
    #dfNUniv.to_csv('aDfnu.csv')
    
    #------- ttest evaluation -----------------------------------------
    #ttest = ttest_ind(dfUniv['price_ratio'],dfNUniv['price_ratio'],nan_policy = 'omit', equal_var='True')
    #print(ttest)
    ttest = ttest_ind(dfUniv['price_ratio'],dfNUniv['price_ratio'], equal_var='True')
    #--- 
    ttestresult = str(ttest[1:2])
    pvalue = float(ttestresult[1:(len(ttestresult)-2)])
    #print(ttest)
    #print(pvalue)
    
    #----- Mean comparison ------------
    if ( dfUniv['price_ratio'].mean() < dfNUniv['price_ratio'].mean() ):
        better = "university town"
    else: 
        better = "non-university town"
        
    #----- Hypothesis Rejection --------------
    if pvalue < 0.01:
        different=True
    else: 
        different=False
        
    '''Return the tuple (different, p, better) where 
    different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). 
    The variable p should be equal to the exact p value returned from scipy.stats.ttest_ind(). 
    The value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivalent to a
    reduced market loss).'''
    
    print(type(different))
    print(type(pvalue))
    print(type(better))
    return (different, pvalue, better)
run_ttest()


/home/latini/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/latini/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/latini/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


<class 'bool'>
<class 'float'>
<class 'str'>


(True, 0.005496427353694603, 'university town')

In [326]:
    ul = get_list_of_university_towns()
    ul.set_index(['State','RegionName'],inplace=True)
    #ul.set_index(['State','RegionName'],drop=True,inplace=True) #Drop!!!
    #print ((ul))#print(df.index.names)
    
    ##ndf=pd.merge(df, ul, how = 'inner', left_index='True', right_on=['State','RegionName'])
    #dfu =pd.merge(ul, df, how = 'left', left_on=['State','RegionName'], right_index='True')
    dfu =pd.merge(ul, df, how = 'left', left_index='True', right_index='True')
    ##print(ndf.info())
    
    print(len(dfu))
    dfnu = hdf.index.difference(dfu.index)#
    #print(hdf.head(15))
    dfnu = pd.DataFrame(index=dfnu)
    
    #dfnu['x']=0
    print(dfu.info()) #.head())
    
    dfnu = pd.merge(dfnu, df, how = 'left', left_index=True, right_index=True)
    print(len(dfnu))
    print(dfnu.info()) #head())
    ''' x '''
    
    #dfnu = hdf.index.difference(dfu.index)#find index values are different 
    #dfnu = pd.DataFrame(index=dfnu)
    #dfnu = pd.merge(dfnu, hdf, how = 'left', left_index=True, right_index=True)###merge to
    
    ##run ttest
    ##ttest = ttest_ind(dfu['price_ratio'],dfnu['price_ratio'],nan_policy = 'omit', equal_var='True')


NameError: name 'ul' is not defined